In [ ]:
#lgb [50]	train's auc: 0.973866	valid's auc: 0.95478
#catboost 50:     learn: 0.9416008        test: 0.9328532 best: 0.9329067 (49)    total: 6m 7s    remaining: 17m 54s
# sample top 4000000 train and val
#catboost, slow..., large memory cost..., low accuracy...

In [33]:
import numpy 
import catboost as cbt
from utils import *
import pandas as pd

In [2]:
df_train, df_val, df_sub = load()

df_train.shape 161570557,13
df_val.shape 23333333,13
df_sub.shape 18790469,13


In [ ]:
sample = 4000000
df_train = df_train[: sample]
df_val = df_val[: sample]

In [3]:
cat_indices = [xnames.index(f) for f in categorical_names]

In [25]:
r = cbt.CatBoostClassifier(
    iterations=200,
    verbose=True,
    thread_count=16, 
    random_seed=2333,
    learning_rate=0.05,
    use_best_model=True,
    depth=5,
    leaf_estimation_method='Gradient',
    eval_metric='AUC',
    custom_metric='AUC',
    scale_pos_weight=99,
)

In [31]:
r.fit(df_train[xnames],
      df_train[yname],
      cat_features=cat_indices,
      use_best_model=True,
      eval_set=[df_val[xnames], df_val[yname]],
      verbose=True,
     ) 

0:	learn: 0.9089927	test: 0.8981687	best: 0.8981687 (0)	total: 1.77s	remaining: 15.9s
1:	learn: 0.9181558	test: 0.9166835	best: 0.9166835 (1)	total: 3.42s	remaining: 13.7s
2:	learn: 0.9262302	test: 0.9210668	best: 0.9210668 (2)	total: 5.13s	remaining: 12s
3:	learn: 0.9281000	test: 0.9211813	best: 0.9211813 (3)	total: 6.87s	remaining: 10.3s
4:	learn: 0.9279648	test: 0.9209041	best: 0.9211813 (3)	total: 8.51s	remaining: 8.51s
5:	learn: 0.9295577	test: 0.9183527	best: 0.9211813 (3)	total: 10.1s	remaining: 6.75s
6:	learn: 0.9296786	test: 0.9183368	best: 0.9211813 (3)	total: 11.7s	remaining: 5.03s
7:	learn: 0.9297945	test: 0.9167141	best: 0.9211813 (3)	total: 13.4s	remaining: 3.34s
8:	learn: 0.9299538	test: 0.9168927	best: 0.9211813 (3)	total: 15s	remaining: 1.67s
9:	learn: 0.9298685	test: 0.9171268	best: 0.9211813 (3)	total: 16.7s	remaining: 0us

bestTest = 0.9211813113
bestIteration = 3

Shrink model to first 4 iterations.


In [10]:
ids = df_sub['click_id'].astype(int).values

In [12]:
print('predicting...')
y_sub = r.predict_proba(df_sub[xnames])[:, 1]

predicting...


In [ ]:
print('writing...')
output = pd.DataFrame({'click_id': ids, 'is_attributed': y_sub})
output.to_csv("../data/cgb.csv", index=False)

writing...
